# budget_tables_draft2017b.ipynb

## Purpose
Produce tables of MARC aerosol budget for *"Effective radiative forcing in the aerosol-climate model CAM5.3-MARC-ARG compared to default CAM5.3"*, based on results from project [p17c-marc-comparison](https://github.com/grandey/p17c-marc-comparison).

## Requirements
- NetCDF files produced by [budget_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/budget_cdo_nco_draft2017b.ipynb) and [analysis_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/analysis_cdo_nco_draft2017b.ipynb).
- Python modules mentioned in cell #2, inlcuding [climapy](https://github.com/grandey/climapy) ([doi:10.5281/zenodo.1053020](https://doi.org/10.5281/zenodo.1053020)).

## Author
Benjamin S. Grandey, 2018

## Acknowledgments
Thanks are due to **Alex Avramov**.  The interpretation of the diagnostics are informed by *print_MARC_budget_Benjamin.pro*, an IDL script written by Alex Avramov.

In [1]:
! date

Thu Aug  2 12:15:04 +08 2018


In [2]:
import climapy
import os
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import xarray as xr

# Print versions of packages
for module in [climapy, np, pd, scipy, xr]:
    try:
        print('{}.__version__ = {}'.format(module.__name__, module.__version__))
    except AttributeError:
        pass
    
# Default precision when displaying DataFrames
pd.set_option('precision', 3)

climapy.__version__ = 0.1.0+b35937c
numpy.__version__ = 1.14.3
pandas.__version__ = 0.23.0
scipy.__version__ = 1.1.0
xarray.__version__ = 0.10.4


## Post-processed data file locations
Location of files written by [budget_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/budget_cdo_nco_draft2017b.ipynb) and [analysis_cdo_nco_draft2017b.ipynb](https://github.com/grandey/p17c-marc-comparison/blob/master/analysis_draft2017b/analysis_cdo_nco_draft2017b.ipynb). The latter contains the column burdens.

In [3]:
budget_dir = os.path.expandvars('$HOME/data/projects/p17c_marc_comparison/budget_cdo_nco_draft2017b/')
analysis_dir = os.path.expandvars('$HOME/data/projects/p17c_marc_comparison/analysis_cdo_nco_draft2017b/')

## Read global totals into dictionary

In [4]:
# Diagnostics to load data for
diagnostic_dict = {
    # Sources
    'c{}_SOURCE': 'Sources, Tg/yr',  # sum of sources
    'c{}_SF': '    Emission',
    'c{}_BNUC': '    Binary nucleation',
    'c{}_CND': '    Condensation',
    'c{}_AGEsrc': '    Aging (source)',
    'c{}_ADJsrc': '    Growth (source)',
    'c{}_CGsrc': '    Coagulation (source)',
    'c{}_EVAP': '    Hydrometeor evaporation',
    # Sinks
    'c{}_SINK': 'Sinks, Tg/yr',  # sum of sinks
    'c{}_AGEsnk': '    Aging (sink)',
    'c{}_ADJsnk': '    Growth (sink)',      
    'c{}_CGsnk': '    Coagulation (sink)',             
    'c{}_ACT': '    Nucleation scavenging by stratiform clouds',
    'c{}_SCV': '    Nucleation scavenging by convective clouds',
    'c{}_IMP': '    Impaction scavenging',
    'c{}_DRY': '    Dry deposition',
    # Burden / loading
    'c{}_LDG': 'Burden, Tg'
}

In [5]:
# List of variables (aerosol-diagnostic combinations) to try loading data for
variable_list = []
for aerosol in ['tSUL', 'tOC', 'tBC', 'tDST', 'tSSLT',
                'NUC', 'AIT', 'ACC', 'OC', 'MOS', 'OIM', 'SIMOS', 'BC', 'MBS', 'BIM', 'SIMBS']:
    for diagnostic in diagnostic_dict.keys():
        variable_list.append(diagnostic.format(aerosol)) 

In [6]:
# Read data and calculate global totals, standard errors
# Initialise dictionaries
mean_dict = {}  # mean global totals
error_dict = {}  # standard errors, based on annual means
# Loop over years and variables
for year in ['2000', '1850']:
    for variable in variable_list:
        variable_year = (variable, year)
        try:
            # Read data from input file
            if '_LDG' in variable:
                in_filename = '{}/marc_s2_{}_{}_ANN.nc'.format(analysis_dir, year, variable)
                in_ds = xr.open_dataset(in_filename, decode_times=False)[variable]
            else:
                in_filename = '{}/marc_s2_{}_{}_ANN.nc'.format(budget_dir, year, variable)
                in_ds = xr.open_dataset(in_filename, decode_times=False)[variable]
            # Exclude first two years as spin-up
            data = in_ds.isel(time=slice(2, None))
            # Calculate sum across earth: kg/m2/s -> kg/s / kg/m2 -> kg
            totals = climapy.xr_area_weighted_stat(data, stat='sum')
            # Convert units
            if '_LDG' in variable:  # kg -> Tg
                totals = totals / 1e9
            else:  # kg/s -> Tg/yr
                totals = totals / 1e9 * 60 * 60 *24 * 365
            # Calculate mean across years
            mean_dict[variable_year] = float(totals.mean())
            # Calculate standard error
            error = totals.std(dim='time', ddof=1).data.flatten() / np.sqrt(totals['time'].size)
            error_dict[variable_year] = float(error)
            # Print mean
            print('{:13}, {}: {:10.4f} ± {:7.4f}'.format(variable, year, mean_dict[variable_year],
                                                        error_dict[variable_year]))
            # Close input file
            in_ds.close()   
        except FileNotFoundError:
            pass
! date

ctSUL_SOURCE , 2000:   538.7005 ±  0.7464
ctSUL_BNUC   , 2000:     0.0019 ±  0.0001
ctSUL_CND    , 2000:    11.5506 ±  0.0531
ctSUL_AGEsrc , 2000:    22.3322 ±  0.0203
ctSUL_EVAP   , 2000:   504.8158 ±  0.7477
ctSUL_SINK   , 2000:  -538.8804 ±  0.7439
ctSUL_ACT    , 2000:  -396.5978 ±  0.6930
ctSUL_SCV    , 2000:   -20.7465 ±  0.0611
ctSUL_IMP    , 2000:  -116.2898 ±  0.0614
ctSUL_DRY    , 2000:    -5.2464 ±  0.0102
ctSUL_LDG    , 2000:     1.3313 ±  0.0028
ctOC_SOURCE  , 2000:   108.8810 ±  0.0000
ctOC_SF      , 2000:   108.8810 ±  0.0000
ctOC_SINK    , 2000:  -108.8222 ±  0.0086
ctOC_ACT     , 2000:   -13.1431 ±  0.0146
ctOC_SCV     , 2000:    -0.5127 ±  0.0021
ctOC_IMP     , 2000:   -90.6522 ±  0.0175
ctOC_DRY     , 2000:    -4.5141 ±  0.0047
ctOC_LDG     , 2000:     1.4934 ±  0.0028
ctBC_SOURCE  , 2000:     7.7587 ±  0.0000
ctBC_SF      , 2000:     7.7587 ±  0.0000
ctBC_SINK    , 2000:    -7.7653 ±  0.0011
ctBC_ACT     , 2000:    -1.0153 ±  0.0019
ctBC_SCV     , 2000:    -0.0277 ± 

cAIT_CND     , 1850:     0.0668 ±  0.0006
cAIT_ADJsrc  , 1850:     0.0058 ±  0.0000
cAIT_CGsrc   , 1850:     0.0000 ±  0.0000
cAIT_SINK    , 1850:    -0.0726 ±  0.0006
cAIT_ADJsnk  , 1850:    -0.0717 ±  0.0006
cAIT_CGsnk   , 1850:    -0.0001 ±  0.0000
cAIT_ACT     , 1850:    -0.0005 ±  0.0000
cAIT_SCV     , 1850:    -0.0000 ±  0.0000
cAIT_IMP     , 1850:    -0.0003 ±  0.0000
cAIT_DRY     , 1850:    -0.0000 ±  0.0000
cAIT_LDG     , 1850:     0.0000 ±  0.0000
cACC_SOURCE  , 1850:   204.4989 ±  0.2879
cACC_CND     , 1850:     3.2114 ±  0.0141
cACC_ADJsrc  , 1850:     0.0717 ±  0.0006
cACC_CGsrc   , 1850:     0.0001 ±  0.0000
cACC_EVAP    , 1850:   201.2157 ±  0.2852
cACC_SINK    , 1850:  -204.5653 ±  0.2887
cACC_CGsnk   , 1850:    -0.2848 ±  0.0020
cACC_ACT     , 1850:  -146.0373 ±  0.2667
cACC_SCV     , 1850:   -10.3152 ±  0.0217
cACC_IMP     , 1850:   -46.0353 ±  0.0175
cACC_DRY     , 1850:    -1.8927 ±  0.0042
cACC_LDG     , 1850:     0.4017 ±  0.0008
cOC_SOURCE   , 1850:    87.9384 ± 

Note: the values are ~0.1% lower than Alex's values, due to using slightly different values for the area of the earth.

## Rearrange into DataFrames

In [7]:
# Initialise dataframes
columns = ['tSUL', 'tOC', 'tBC', 'tDST', 'tSSLT',
           'NUC', 'AIT', 'ACC', 'OC', 'MOS', 'OIM', 'SIMOS', 'BC', 'MBS', 'BIM', 'SIMBS']
mean_2000_df = pd.DataFrame(columns=columns)  # marc_s2_2000
error_2000_df = pd.DataFrame(columns=columns)
mean_1850_df = pd.DataFrame(columns=columns)  # marc_s2_1850
error_1850_df = pd.DataFrame(columns=columns)

In [8]:
# Populate with data for diagnostics
for k, v in diagnostic_dict.items():  # loop over diagnostics
    for col in columns:  # loop over columns
        if (k.format(col), '2000') in mean_dict:
            mean_2000_df.loc[v, col] = mean_dict[(k.format(col), '2000')]
            error_2000_df.loc[v, col] = error_dict[(k.format(col), '2000')]
            mean_1850_df.loc[v, col] = mean_dict[(k.format(col), '1850')]
            error_1850_df.loc[v, col] = error_dict[(k.format(col), '1850')]

In [9]:
# Lifetime
mean_2000_df.loc['Lifetime, days'] = (mean_2000_df.loc['Burden, Tg'] /
                                      mean_2000_df.loc['Sinks, Tg/yr'].abs() * 365)
error_2000_df.loc['Lifetime, days'] = (mean_2000_df.loc['Lifetime, days'] *
                                       ((error_2000_df.loc['Burden, Tg'] /
                                         mean_2000_df.loc['Burden, Tg'])**2 +
                                        (error_2000_df.loc['Sinks, Tg/yr'] /
                                         mean_2000_df.loc['Sinks, Tg/yr'].abs())**2).pow(0.5))
mean_1850_df.loc['Lifetime, days'] = (mean_1850_df.loc['Burden, Tg'] /
                                      mean_1850_df.loc['Sinks, Tg/yr'] * -365)
error_1850_df.loc['Lifetime, days'] = (mean_1850_df.loc['Lifetime, days'] *
                                       ((error_1850_df.loc['Burden, Tg'] /
                                         mean_1850_df.loc['Burden, Tg'])**2 +
                                        (error_1850_df.loc['Sinks, Tg/yr'] /
                                         mean_1850_df.loc['Sinks, Tg/yr'].abs())**2).pow(0.5))

In [10]:
# Print means for marc_s2_2000
mean_2000_df

,tSUL,tOC,tBC,tDST,tSSLT,NUC,AIT,ACC,OC,MOS,OIM,SIMOS,BC,MBS,BIM,SIMBS
"Sources, Tg/yr",539,109,7.76,3.68e+03,5.48e+03,0.00908,0.0911,509,109,50.3,23.8,26.4,7.76,8.3,1.83,6.46
Emission,NaN,109,7.76,3.68e+03,5.48e+03,NaN,NaN,NaN,109,NaN,NaN,NaN,7.76,NaN,NaN,NaN
Binary nucleation,0.00195,NaN,NaN,NaN,NaN,0.00195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Condensation,11.6,NaN,NaN,NaN,NaN,0.00714,0.0841,4.37,NaN,3.84,NaN,3.84,NaN,3.25,NaN,3.25
Aging (source),22.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.2,21.6,20.6,NaN,3.58,1.83,1.75
Growth (source),NaN,NaN,NaN,NaN,NaN,NaN,0.00695,0.0897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Coagulation (source),NaN,NaN,NaN,NaN,NaN,NaN,1.54e-05,0.000115,NaN,4.24,2.22,2.02,NaN,1.47,NaN,1.47
Hydrometeor evaporation,505,NaN,NaN,NaN,NaN,NaN,NaN,505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Sinks, Tg/yr",-539,-109,-7.77,-3.71e+03,-5.53e+03,-0.0091,-0.0911,-509,-109,-50.4,-23.9,-26.5,-7.76,-8.32,-1.84,-6.48
Aging (sink),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-21.6,NaN,NaN,NaN,-1.83,NaN,NaN,NaN


In [11]:
# Print means for marc_s2_1850
mean_1850_df

,tSUL,tOC,tBC,tDST,tSSLT,NUC,AIT,ACC,OC,MOS,OIM,SIMOS,BC,MBS,BIM,SIMBS
"Sources, Tg/yr",211,87.9,3.08,3.79e+03,5.51e+03,0.00665,0.0726,204,87.9,12.4,6.15,6.26,3.08,0.715,0.159,0.556
Emission,NaN,87.9,3.08,3.79e+03,5.51e+03,NaN,NaN,NaN,87.9,NaN,NaN,NaN,3.08,NaN,NaN,NaN
Binary nucleation,0.000882,NaN,NaN,NaN,NaN,0.000882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Condensation,4.57,NaN,NaN,NaN,NaN,0.00576,0.0668,3.21,NaN,0.969,NaN,0.969,NaN,0.322,NaN,0.322
Aging (source),5.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.4,5.34,5.09,NaN,0.31,0.159,0.151
Growth (source),NaN,NaN,NaN,NaN,NaN,NaN,0.00583,0.0717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Coagulation (source),NaN,NaN,NaN,NaN,NaN,NaN,1.36e-05,9.55e-05,NaN,1.01,0.81,0.202,NaN,0.0825,NaN,0.0825
Hydrometeor evaporation,201,NaN,NaN,NaN,NaN,NaN,NaN,201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Sinks, Tg/yr",-211,-87.9,-3.09,-3.81e+03,-5.56e+03,-0.00665,-0.0726,-205,-87.9,-12.5,-6.17,-6.29,-3.09,-0.718,-0.16,-0.558
Aging (sink),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.34,NaN,NaN,NaN,-0.159,NaN,NaN,NaN


## Sulfate tables

In [12]:
# Year-2000
# Sulfate columns
sul_cols = ['tSUL', 'NUC', 'AIT', 'ACC', 'SIMOS', 'SIMBS',]
# Extract sulfate data
sul_mean_2000_df = mean_2000_df[sul_cols]
sul_error_2000_df = error_2000_df[sul_cols]
# Drop rows with no data
sul_mean_2000_df = sul_mean_2000_df.dropna(how='all')
sul_error_2000_df = sul_error_2000_df.dropna(how='all')
# Convert values to strings
sul_mean_2000_df = sul_mean_2000_df.applymap(lambda x: '%+.2f' % x)
sul_error_2000_df = sul_error_2000_df.applymap(lambda x: '%.2f' % x)
# Combine mean and standard error strings
sul_2000_df = sul_mean_2000_df + ' ± ' + sul_error_2000_df
# Replace 'nan ± nan' with ''
sul_2000_df = sul_2000_df.replace('nan ± nan', '')
sul_2000_df = sul_2000_df.replace('+nan ± nan', '')
# Rename columns
sul_2000_df = sul_2000_df.rename(index=str, columns={'tSUL': 'Sulfate aerosol',
                                                     'NUC': 'in NUC',
                                                     'AIT': 'in AIT',
                                                     'ACC': 'in ACC',
                                                     'SIMOS': 'in MOS',
                                                     'SIMBS': 'in MBS'})
# Replace Tg with Tg(SO4)
for i in sul_2000_df.index:  # not an efficient method, but it works!
    if 'Tg' in i:
        sul_2000_df = sul_2000_df.rename({i: 'Tg(SO4)'.join(i.split('Tg'))})
# Save to CSV
out_filename = 'csv/t01.csv'
sul_2000_df.to_csv(out_filename)
print('Written {}'.format(out_filename))
# Save to Excel
out_filename = 'csv/t01.xlsx'
sul_2000_df.to_excel(out_filename)
print('Written {}'.format(out_filename))
# Display
sul_2000_df.style.set_caption('Year-2000')

Written csv/t01.csv
Written csv/t01.xlsx


,Sulfate aerosol,in NUC,in AIT,in ACC,in MOS,in MBS
"Sources, Tg(SO4)/yr",+538.70 ± 0.75,+0.01 ± 0.00,+0.09 ± 0.00,+509.28 ± 0.75,+26.45 ± 0.04,+6.46 ± 0.03
Binary nucleation,+0.00 ± 0.00,+0.00 ± 0.00,,,,
Condensation,+11.55 ± 0.05,+0.01 ± 0.00,+0.08 ± 0.00,+4.37 ± 0.01,+3.84 ± 0.02,+3.25 ± 0.02
Aging (source),+22.33 ± 0.02,,,,+20.59 ± 0.02,+1.75 ± 0.00
Growth (source),,,+0.01 ± 0.00,+0.09 ± 0.00,,
Coagulation (source),,,+0.00 ± 0.00,+0.00 ± 0.00,+2.02 ± 0.01,+1.47 ± 0.01
Hydrometeor evaporation,+504.82 ± 0.75,,,+504.82 ± 0.75,,
"Sinks, Tg(SO4)/yr",-538.88 ± 0.74,-0.01 ± 0.00,-0.09 ± 0.00,-509.37 ± 0.75,-26.52 ± 0.04,-6.48 ± 0.03
Growth (sink),,-0.01 ± 0.00,-0.09 ± 0.00,,,
Coagulation (sink),,-0.00 ± 0.00,-0.00 ± 0.00,-3.49 ± 0.02,,


In [13]:
# Year-1850
# Extract sulfate data
sul_mean_1850_df = mean_1850_df[sul_cols]
sul_error_1850_df = error_1850_df[sul_cols]
# Drop rows with no data
sul_mean_1850_df = sul_mean_1850_df.dropna(how='all')
sul_error_1850_df = sul_error_1850_df.dropna(how='all')
# Convert values to strings
sul_mean_1850_df = sul_mean_1850_df.applymap(lambda x: '%+.2f' % x)
sul_error_1850_df = sul_error_1850_df.applymap(lambda x: '%.2f' % x)
# Combine mean and standard error strings
sul_1850_df = sul_mean_1850_df + ' ± ' + sul_error_1850_df
# Replace 'nan ± nan' with ''
sul_1850_df = sul_1850_df.replace('nan ± nan', '')
sul_1850_df = sul_1850_df.replace('+nan ± nan', '')
# Rename columns
sul_1850_df = sul_1850_df.rename(index=str, columns={'tSUL': 'Sulfate aerosol',
                                                     'NUC': 'in NUC',
                                                     'AIT': 'in AIT',
                                                     'ACC': 'in ACC',
                                                     'SIMOS': 'in MOS',
                                                     'SIMBS': 'in MBS'})
# Replace Tg with Tg(SO4)
for i in sul_1850_df.index:
    if 'Tg' in i:
        sul_1850_df = sul_1850_df.rename({i: 'Tg(SO4)'.join(i.split('Tg'))})
# Save to CSV
out_filename = 'csv/tS01.csv'
sul_1850_df.to_csv(out_filename)
print('Written {}'.format(out_filename))
# Save to Excel
out_filename = 'csv/tS01.xlsx'
sul_1850_df.to_excel(out_filename)
print('Written {}'.format(out_filename))
# Display
sul_1850_df.style.set_caption('Year-1850')

Written csv/tS01.csv
Written csv/tS01.xlsx


,Sulfate aerosol,in NUC,in AIT,in ACC,in MOS,in MBS
"Sources, Tg(SO4)/yr",+211.03 ± 0.29,+0.01 ± 0.00,+0.07 ± 0.00,+204.50 ± 0.29,+6.26 ± 0.02,+0.56 ± 0.01
Binary nucleation,+0.00 ± 0.00,+0.00 ± 0.00,,,,
Condensation,+4.57 ± 0.02,+0.01 ± 0.00,+0.07 ± 0.00,+3.21 ± 0.01,+0.97 ± 0.01,+0.32 ± 0.01
Aging (source),+5.24 ± 0.01,,,,+5.09 ± 0.01,+0.15 ± 0.00
Growth (source),,,+0.01 ± 0.00,+0.07 ± 0.00,,
Coagulation (source),,,+0.00 ± 0.00,+0.00 ± 0.00,+0.20 ± 0.00,+0.08 ± 0.00
Hydrometeor evaporation,+201.22 ± 0.29,,,+201.22 ± 0.29,,
"Sinks, Tg(SO4)/yr",-211.13 ± 0.29,-0.01 ± 0.00,-0.07 ± 0.00,-204.57 ± 0.29,-6.29 ± 0.02,-0.56 ± 0.01
Growth (sink),,-0.01 ± 0.00,-0.07 ± 0.00,,,
Coagulation (sink),,-0.00 ± 0.00,-0.00 ± 0.00,-0.28 ± 0.00,,


## Organic carbon (OC) tables

In [14]:
# Year-2000
# OC columns
oc_cols = ['tOC', 'OC', 'OIM',]
# Extract OC data
oc_mean_2000_df = mean_2000_df[oc_cols]
oc_error_2000_df = error_2000_df[oc_cols]
# Drop rows with no data
oc_mean_2000_df = oc_mean_2000_df.dropna(how='all')
oc_error_2000_df = oc_error_2000_df.dropna(how='all')
# Convert values to strings
oc_mean_2000_df = oc_mean_2000_df.applymap(lambda x: '%+.2f' % x)
oc_error_2000_df = oc_error_2000_df.applymap(lambda x: '%.2f' % x)
# Combine mean and standard error strings
oc_2000_df = oc_mean_2000_df + ' ± ' + oc_error_2000_df
# Replace 'nan ± nan' with ''
oc_2000_df = oc_2000_df.replace('nan ± nan', '')
oc_2000_df = oc_2000_df.replace('+nan ± nan', '')
# Rename columns
oc_2000_df = oc_2000_df.rename(index=str, columns={'tOC': 'Organic carbon aerosol',
                                                   'OC': 'in pure OC',
                                                   'OIM': 'in MOS'})
# Save to CSV
out_filename = 'csv/t02.csv'
oc_2000_df.to_csv(out_filename)
print('Written {}'.format(out_filename))
# Save to Excel
out_filename = 'csv/t02.xlsx'
oc_2000_df.to_excel(out_filename)
print('Written {}'.format(out_filename))
# Display
oc_2000_df.style.set_caption('Year-2000')

Written csv/t02.csv
Written csv/t02.xlsx


,Organic carbon aerosol,in pure OC,in MOS
"Sources, Tg/yr",+108.88 ± 0.00,+108.88 ± 0.00,+23.81 ± 0.02
Emission,+108.88 ± 0.00,+108.88 ± 0.00,
Aging (source),,,+21.60 ± 0.02
Coagulation (source),,,+2.22 ± 0.01
"Sinks, Tg/yr",-108.82 ± 0.01,-108.75 ± 0.01,-23.88 ± 0.02
Aging (sink),,-21.60 ± 0.02,
Coagulation (sink),,-2.22 ± 0.01,
Nucleation scavenging by stratiform clouds,-13.14 ± 0.01,-0.00 ± 0.00,-13.14 ± 0.01
Nucleation scavenging by convective clouds,-0.51 ± 0.00,-0.00 ± 0.00,-0.51 ± 0.00
Impaction scavenging,-90.65 ± 0.02,-81.03 ± 0.02,-9.63 ± 0.02


In [15]:
# Year-1850
# OC columns
oc_cols = ['tOC', 'OC', 'OIM',]
# Extract OC data
oc_mean_1850_df = mean_1850_df[oc_cols]
oc_error_1850_df = error_1850_df[oc_cols]
# Drop rows with no data
oc_mean_1850_df = oc_mean_1850_df.dropna(how='all')
oc_error_1850_df = oc_error_1850_df.dropna(how='all')
# Convert values to strings
oc_mean_1850_df = oc_mean_1850_df.applymap(lambda x: '%+.2f' % x)
oc_error_1850_df = oc_error_1850_df.applymap(lambda x: '%.2f' % x)
# Combine mean and standard error strings
oc_1850_df = oc_mean_1850_df + ' ± ' + oc_error_1850_df
# Replace 'nan ± nan' with ''
oc_1850_df = oc_1850_df.replace('nan ± nan', '')
oc_1850_df = oc_1850_df.replace('+nan ± nan', '')
# Rename columns
oc_1850_df = oc_1850_df.rename(index=str, columns={'tOC': 'Organic carbon aerosol',
                                                   'OC': 'in pure OC',
                                                   'OIM': 'in MOS'})
# Save to CSV
out_filename = 'csv/tS02.csv'
oc_1850_df.to_csv(out_filename)
print('Written {}'.format(out_filename))
# Save to Excel
out_filename = 'csv/tS02.xlsx'
oc_1850_df.to_excel(out_filename)
print('Written {}'.format(out_filename))
# Display
oc_1850_df.style.set_caption('Year-1850')

Written csv/tS02.csv
Written csv/tS02.xlsx


,Organic carbon aerosol,in pure OC,in MOS
"Sources, Tg/yr",+87.94 ± 0.00,+87.94 ± 0.00,+6.15 ± 0.01
Emission,+87.94 ± 0.00,+87.94 ± 0.00,
Aging (source),,,+5.34 ± 0.01
Coagulation (source),,,+0.81 ± 0.00
"Sinks, Tg/yr",-87.90 ± 0.01,-87.87 ± 0.01,-6.17 ± 0.01
Aging (sink),,-5.34 ± 0.01,
Coagulation (sink),,-0.81 ± 0.00,
Nucleation scavenging by stratiform clouds,-3.32 ± 0.01,-0.00 ± 0.00,-3.32 ± 0.01
Nucleation scavenging by convective clouds,-0.22 ± 0.00,+0.00 ± 0.00,-0.22 ± 0.00
Impaction scavenging,-80.68 ± 0.01,-78.16 ± 0.01,-2.52 ± 0.01


## Black carbon (BC) tables

In [16]:
# Year-2000
# BC columns
bc_cols = ['tBC', 'BC', 'BIM',]
# Extract BC data
bc_mean_2000_df = mean_2000_df[bc_cols]
bc_error_2000_df = error_2000_df[bc_cols]
# Drop rows with no data
bc_mean_2000_df = bc_mean_2000_df.dropna(how='all')
bc_error_2000_df = bc_error_2000_df.dropna(how='all')
# Convert values to strings
bc_mean_2000_df = bc_mean_2000_df.applymap(lambda x: '%+.2f' % x)
bc_error_2000_df = bc_error_2000_df.applymap(lambda x: '%.2f' % x)
# Combine mean and standard error strings
bc_2000_df = bc_mean_2000_df + ' ± ' + bc_error_2000_df
# Replace 'nan ± nan' with ''
bc_2000_df = bc_2000_df.replace('nan ± nan', '')
bc_2000_df = bc_2000_df.replace('+nan ± nan', '')
# Rename columns
bc_2000_df = bc_2000_df.rename(index=str, columns={'tBC': 'Black carbon aerosol',
                                                   'BC': 'in pure BC',
                                                   'BIM': 'in MBS'})
# Save to CSV
out_filename = 'csv/t03.csv'
bc_2000_df.to_csv(out_filename)
print('Written {}'.format(out_filename))
# Save to Excel
out_filename = 'csv/t03.xlsx'
bc_2000_df.to_excel(out_filename)
print('Written {}'.format(out_filename))
# Display
bc_2000_df.style.set_caption('Year-2000')

Written csv/t03.csv
Written csv/t03.xlsx


,Black carbon aerosol,in pure BC,in MBS
"Sources, Tg/yr",+7.76 ± 0.00,+7.76 ± 0.00,+1.83 ± 0.00
Emission,+7.76 ± 0.00,+7.76 ± 0.00,
Aging (source),,,+1.83 ± 0.00
"Sinks, Tg/yr",-7.77 ± 0.00,-7.76 ± 0.00,-1.84 ± 0.00
Aging (sink),,-1.83 ± 0.00,
Nucleation scavenging by stratiform clouds,-1.02 ± 0.00,-0.00 ± 0.00,-1.02 ± 0.00
Nucleation scavenging by convective clouds,-0.03 ± 0.00,-0.00 ± 0.00,-0.03 ± 0.00
Impaction scavenging,-6.31 ± 0.00,-5.56 ± 0.00,-0.74 ± 0.00
Dry deposition,-0.42 ± 0.00,-0.36 ± 0.00,-0.05 ± 0.00
"Burden, Tg",+0.11 ± 0.00,+0.09 ± 0.00,+0.02 ± 0.00


In [17]:
# Year-1850
# BC columns
bc_cols = ['tBC', 'BC', 'BIM',]
# Extract BC data
bc_mean_1850_df = mean_1850_df[bc_cols]
bc_error_1850_df = error_1850_df[bc_cols]
# Drop rows with no data
bc_mean_1850_df = bc_mean_1850_df.dropna(how='all')
bc_error_1850_df = bc_error_1850_df.dropna(how='all')
# Convert values to strings
bc_mean_1850_df = bc_mean_1850_df.applymap(lambda x: '%+.2f' % x)
bc_error_1850_df = bc_error_1850_df.applymap(lambda x: '%.2f' % x)
# Combine mean and standard error strings
bc_1850_df = bc_mean_1850_df + ' ± ' + bc_error_1850_df
# Replace 'nan ± nan' with ''
bc_1850_df = bc_1850_df.replace('nan ± nan', '')
bc_1850_df = bc_1850_df.replace('+nan ± nan', '')
# Rename columns
bc_1850_df = bc_1850_df.rename(index=str, columns={'tBC': 'Black carbon aerosol',
                                                   'BC': 'in pure BC',
                                                   'BIM': 'in MBS'})
# Save to CSV
out_filename = 'csv/tS03.csv'
bc_1850_df.to_csv(out_filename)
print('Written {}'.format(out_filename))
# Save to Excel
out_filename = 'csv/tS03.xlsx'
bc_1850_df.to_excel(out_filename)
print('Written {}'.format(out_filename))
# Display
bc_1850_df.style.set_caption('Year-1850')

Written csv/tS03.csv
Written csv/tS03.xlsx


,Black carbon aerosol,in pure BC,in MBS
"Sources, Tg/yr",+3.08 ± 0.00,+3.08 ± 0.00,+0.16 ± 0.00
Emission,+3.08 ± 0.00,+3.08 ± 0.00,
Aging (source),,,+0.16 ± 0.00
"Sinks, Tg/yr",-3.09 ± 0.00,-3.09 ± 0.00,-0.16 ± 0.00
Aging (sink),,-0.16 ± 0.00,
Nucleation scavenging by stratiform clouds,-0.08 ± 0.00,-0.00 ± 0.00,-0.08 ± 0.00
Nucleation scavenging by convective clouds,-0.00 ± 0.00,-0.00 ± 0.00,-0.00 ± 0.00
Impaction scavenging,-2.81 ± 0.00,-2.74 ± 0.00,-0.07 ± 0.00
Dry deposition,-0.19 ± 0.00,-0.18 ± 0.00,-0.00 ± 0.00
"Burden, Tg",+0.05 ± 0.00,+0.05 ± 0.00,+0.00 ± 0.00


## Dust and sea-salt

In [18]:
# Year-2000
# Columns
dust_salt_cols = ['tDST', 'tSSLT']
# Extract data
dust_salt_mean_2000_df = mean_2000_df[dust_salt_cols]
dust_salt_error_2000_df = error_2000_df[dust_salt_cols]
# Drop rows with no data
dust_salt_mean_2000_df = dust_salt_mean_2000_df.dropna(how='all')
dust_salt_error_2000_df = dust_salt_error_2000_df.dropna(how='all')
# Convert values to strings
dust_salt_mean_2000_df = dust_salt_mean_2000_df.applymap(lambda x: '%+.2f' % x)
dust_salt_error_2000_df = dust_salt_error_2000_df.applymap(lambda x: '%.2f' % x)
# Combine mean and standard error strings
dust_salt_2000_df = dust_salt_mean_2000_df + ' ± ' + dust_salt_error_2000_df
# Replace 'nan ± nan' with ''
dust_salt_2000_df = dust_salt_2000_df.replace('nan ± nan', '')
dust_salt_2000_df = dust_salt_2000_df.replace('+nan ± nan', '')
# Rename columns
dust_salt_2000_df = dust_salt_2000_df.rename(index=str, columns={'tSSLT': 'Sea salt aerosol',
                                                                 'tDST': 'Dust aerosol',})
# Save to CSV
out_filename = 'csv/t04.csv'
dust_salt_2000_df.to_csv(out_filename)
print('Written {}'.format(out_filename))
# Save to Excel
out_filename = 'csv/t04.xlsx'
dust_salt_2000_df.to_excel(out_filename)
print('Written {}'.format(out_filename))
# Display
dust_salt_2000_df.style.set_caption('Year-2000')

Written csv/t04.csv
Written csv/t04.xlsx


,Dust aerosol,Sea salt aerosol
"Sources, Tg/yr",+3683.19 ± 25.59,+5484.88 ± 11.18
Emission,+3683.19 ± 25.59,+5484.88 ± 11.18
"Sinks, Tg/yr",-3705.79 ± 25.88,-5533.62 ± 11.24
Impaction scavenging,-1819.27 ± 11.26,-2324.42 ± 4.29
Dry deposition,-1886.52 ± 16.05,-3209.19 ± 7.27
"Burden, Tg",+40.91 ± 0.30,+9.60 ± 0.02
"Lifetime, days",+4.03 ± 0.04,+0.63 ± 0.00


In [19]:
# Year-1850
# Columns
dust_salt_cols = ['tSSLT', 'tDST']
# Extract data
dust_salt_mean_1850_df = mean_1850_df[dust_salt_cols]
dust_salt_error_1850_df = error_1850_df[dust_salt_cols]
# Drop rows with no data
dust_salt_mean_1850_df = dust_salt_mean_1850_df.dropna(how='all')
dust_salt_error_1850_df = dust_salt_error_1850_df.dropna(how='all')
# Convert values to strings
dust_salt_mean_1850_df = dust_salt_mean_1850_df.applymap(lambda x: '%+.2f' % x)
dust_salt_error_1850_df = dust_salt_error_1850_df.applymap(lambda x: '%.2f' % x)
# Combine mean and standard error strings
dust_salt_1850_df = dust_salt_mean_1850_df + ' ± ' + dust_salt_error_1850_df
# Replace 'nan ± nan' with ''
dust_salt_1850_df = dust_salt_1850_df.replace('nan ± nan', '')
dust_salt_1850_df = dust_salt_1850_df.replace('+nan ± nan', '')
# Rename columns
dust_salt_1850_df = dust_salt_1850_df.rename(index=str, columns={'tSSLT': 'Sea salt aerosol',
                                                                 'tDST': 'Dust aerosol',})
# Save to CSV
out_filename = 'csv/tS04.csv'
dust_salt_1850_df.to_csv(out_filename)
print('Written {}'.format(out_filename))
# Save to Excel
out_filename = 'csv/tS04.xlsx'
dust_salt_1850_df.to_excel(out_filename)
print('Written {}'.format(out_filename))
# Display
dust_salt_1850_df.style.set_caption('Year-1850')

Written csv/tS04.csv
Written csv/tS04.xlsx


,Sea salt aerosol,Dust aerosol
"Sources, Tg/yr",+5510.06 ± 14.81,+3789.26 ± 21.87
Emission,+5510.06 ± 14.81,+3789.26 ± 21.87
"Sinks, Tg/yr",-5559.09 ± 14.96,-3812.70 ± 21.36
Impaction scavenging,-2327.11 ± 5.73,-1898.77 ± 8.88
Dry deposition,-3231.97 ± 9.45,-1913.94 ± 14.32
"Burden, Tg",+9.66 ± 0.02,+41.83 ± 0.24
"Lifetime, days",+0.63 ± 0.00,+4.00 ± 0.03


In [20]:
! date

Thu Aug  2 12:15:12 +08 2018
